In [1]:
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC 
import pandas as pd
import numpy as np
import random
import pickle

import warnings
warnings.filterwarnings("ignore")

In [2]:
# load the data
df_rna = pd.read_hdf("rna_scaled.hdf")
df_dnase = pd.read_hdf("dnase_scaled.hdf")
df_gene_info = pd.read_hdf("df_gene_info.hdf")

In [3]:
# a find gene list function that accept a Dnase position, a df_RNA file, a df_gene_info data frame, and a distance number, and then
# return the list of gene names for prediction and the matrix of gene expression
def find_gene_list(dnase_position, df_gene_info, distance):
    dnase_ls = dnase_position.split("-")
    chr_id = dnase_ls[0]
    chr_start = int(dnase_ls[1])
    chr_end = int(dnase_ls[2])
    
    # define the interval for gene selection
    gene_chr = chr_id.replace("chr", "")
    gene_start = chr_start - distance
    gene_end = chr_end + distance
    #print(gene_chr, gene_start, gene_end)
    # find the gene list for the gene interval
    df_gene_filter = df_gene_info[df_gene_info["chr"] == gene_chr]
    df_gene_filter = df_gene_filter[df_gene_filter["start"] >= gene_start]
    df_gene_filter = df_gene_filter[df_gene_filter["start"] <= gene_end]
    gene_ls = df_gene_filter.index
    return(gene_ls)

In [4]:
# a combined data set function, that accept a df_RNA data set and a df_dnase data set and then combine the two data set,
# and return the columns for the RNA genes and Dnase set
def combine_RNA_dnase(df_rna, df_dnase):
    df_rna["tissue"] = df_rna.index
    df_dnase["tissue"] = df_dnase.index
    df_combine = pd.merge(df_rna, df_dnase, on="tissue")
    df_combine.index = df_rna.index
    gene_list = df_rna.loc[:, df_rna.columns != "tissue"].columns
    dnase_list = df_dnase.loc[:, df_dnase.columns != "tissue"].columns
    return (df_combine, gene_list, dnase_list)

In [5]:
# Here I define a function that accept a Dnase id, df_Dnase and df_RNA combined data set，
# df_gene_info, a distance for computing nearest genes and return the X matrix and Y vector.
def data_matrix(Dnase_id, df_combine, df_gene_info, distance):
    
    # find the nearest gene list
    nearest_genes = list(find_gene_list(Dnase_id,df_gene_info, distance))
    
    # extract and return the df_dnase vector and df_rna matrix
    df_combine[Dnase_id][df_combine[Dnase_id] != 0] = 1 # the value of predict_y is 0 or 1， representing close/open
    pred_y = df_combine[Dnase_id]
    pred_x = df_combine[nearest_genes]
    return((pred_x, pred_y))

In [6]:
# define a function that return mse based on random forest
def mse_random_forest(pred_x, pred_y):
    rf = RandomForestClassifier(n_estimators = 100, n_jobs=-1)
    pred = cross_val_predict(rf, np.array(pred_x), np.array(pred_y), cv = 10, n_jobs=-1)
    return(mean_squared_error(pred_y, pred))

In [7]:
Dnase_id = df_dnase.columns[1]
df_combine = combine_RNA_dnase(df_rna, df_dnase)[0]
distance = 1000000
pred_x = data_matrix(Dnase_id, df_combine, df_gene_info, distance)[0]
pred_y = data_matrix(Dnase_id, df_combine, df_gene_info, distance)[1]

In [8]:
mse_random_forest(pred_x, pred_y)

0.31578947368421051

In [9]:
# define a function that return mse based on support vector machine
def mse_svc(pred_x, pred_y):
    svc = SVC()
    pred = cross_val_predict(svc, np.array(pred_x), np.array(pred_y), cv = 10, n_jobs=-1)
    #mse_ls.append(mean_squared_error(pred_y, pred))
    
    return(mean_squared_error(pred_y, pred))

In [10]:
mse_svc(pred_x, pred_y)

0.36842105263157893

In [11]:
# this function return a mse based on average prediction
def mse_aver(pred_x, pred_y):
    return(np.mean((pred_y -  np.mean(pred_y))**2))

In [12]:
mse_aver(pred_x, pred_y)

0.2160664819944598

In [13]:
# The function accept a list of Dnase site, and then compute the mse score for different 
# prediction models, and return a data frame for the score for different models
def mse_models(dnase_list, df_combine, df_gene_info, distance, output_folder):
    # define the mse list for different models
    rf = []
    svc = []
    ave = []
    # for each dnase site, record the mse of the model
    for dnase_id in dnase_list:
        print("Start calculating Dnase site: ", dnase_id)
        pred_x, pred_y = data_matrix(dnase_id, df_combine, df_gene_info, distance)
        # there are nearby genes
        if pred_x.shape[1] != 0:
            print("Calculate random forest mse...")
            rf.append(mse_random_forest(pred_x, pred_y))
            print("Calculate svc mse...")
            svr.append(mse_svc(pred_x, pred_y))
            print("Calculate the average mse...")
            ave.append(mse_aver(pred_x, pred_y))
        # if there are no nearby genes, then use the average score for prediction
        else:
            mse_ave = np.mean((pred_y -  np.mean(pred_y))**2)
            rf.append(mse_ave)
            svc.append(mse_ave)
            ave.append(mse_ave)
            
    result = pd.DataFrame({"rf" : rf,"svc" : svc, "ave" : ave})
    
    folder = output_folder
    if not os.path.exists(folder):
        os.makedirs(folder)
        
    result.to_csv(folder + "/compare_mse_models.csv")
    return(0)

In [ ]:
# here I am going to sample 1000 dnase site and then compute the mse with different scores

# load the data
df_rna = pd.read_hdf("rna_scaled.hdf")
df_dnase = pd.read_hdf("dnase_scaled.hdf")
df_gene_info = pd.read_hdf("df_gene_info.hdf")
#output_folder = "./result/result_training"
output_folder = "./Results_clf"

df_combine, gene_list, dnase_list = combine_RNA_dnase(df_rna, df_dnase)
sample_number = 10
distance = 1000000

dnase_list_sample = random.sample(list(dnase_list), sample_number)

train_list = random.sample(list(df_combine.index), 16)

test_list = list(set(list(df_combine.index)).difference(set(train_list)))

dnase_list_file = output_folder + "/dnase_list.pickle"
train_list_file = output_folder + "/train_list.pickle"
test_list_file = output_folder + "/test_list.pickle"
data_file = output_folder + "/combined_data.pickle"

with open(dnase_list_file, "wb") as fp:
    pickle.dump(dnase_list_sample, fp)

with open(train_list_file, "wb") as fp:
    pickle.dump(train_list, fp)
    
with open(test_list_file, "wb") as fp:
    pickle.dump(test_list, fp)

with open(data_file, "wb") as fp:
    pickle.dump(df_combine, fp)
print("Tissue used for training: ", train_list)
print("Tissue used for testing: ", test_list)

In [ ]:
mse_models(dnase_list_sample, df_combine.loc[train_list,:], df_gene_info, distance, output_folder)